In [1]:
!pip install chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.3 MB/s eta 

In [2]:
!pip install -U langchain
!pip install --upgrade pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.1
    Uninstalling langchain-core-1.1.1:
      Successfully uninstalled langchain-core-1.1.1
  Attempting uninstall: langchain
    Found existing installation: langchain 1.1.2
    Uninstalling langchain-1.1.2:
      Successfully uninstalled langchain-1.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 79.9 MB/s eta 0:00:00


In [3]:
!pip install langchain-text-splitters

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [5]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain_text_splitters import CharacterTextSplitter
import langchain
from PyPDF2 import PdfReader
import langchain_text_splitters

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pdf_path = r'/content/drive/MyDrive/Hugging_face/льготы-СВО-обновлённая-версия.pdf'

In [8]:
# Шаг 1: Извлечение текста
print("📖 Читаю PDF-файл...")
reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text() or ""

📖 Читаю PDF-файл...


In [9]:
# Шаг 2: Разделение на фрагменты
print("✂️ Разделяю текст на фрагменты...")
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.split_text(text)
print(f"✅ Создано {len(chunks)} фрагментов")

✂️ Разделяю текст на фрагменты...
✅ Создано 41 фрагментов


In [ ]:
embedding_function = []
client = []
collection = []

In [ ]:
# 3. Настройка Chroma и модели эмбеддингов
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
client = chromadb.PersistentClient(path="./chroma_db5")  # Данные сохранятся на диск
collection = client.create_collection(name="my_documents5", embedding_function=embedding_function)

print("📥 Добавляю фрагменты в векторную базу данных...")
for i, chunk in enumerate(chunks):
    # Важно: пропускаем пустые фрагменты
    if chunk.strip():
        collection.add(
            documents=[chunk],  # Передаем список из одного документа
            metadatas=[{"source": pdf_path, "chunk_id": i, "total_chunks": len(chunks)}],
            ids=[f"chunk_{i}"]
        )
print(f"✅ В базу добавлено {collection.count()} фрагментов")

📥 Добавляю фрагменты в векторную базу данных...
✅ В базу добавлено 41 фрагментов


In [ ]:
print(collection)

Collection(name=my_documents)


In [ ]:
collection.get()

{'ids': ['chunk_0',
  'chunk_1',
  'chunk_2',
  'chunk_3',
  'chunk_4',
  'chunk_5',
  'chunk_6',
  'chunk_7',
  'chunk_8',
  'chunk_9',
  'chunk_10',
  'chunk_11',
  'chunk_12',
  'chunk_13',
  'chunk_14',
  'chunk_15',
  'chunk_16',
  'chunk_17',
  'chunk_18',
  'chunk_19',
  'chunk_20',
  'chunk_21',
  'chunk_22',
  'chunk_23',
  'chunk_24',
  'chunk_25',
  'chunk_26',
  'chunk_27',
  'chunk_28',
  'chunk_29',
  'chunk_30',
  'chunk_31',
  'chunk_32',
  'chunk_33',
  'chunk_34',
  'chunk_35',
  'chunk_36',
  'chunk_37',
  'chunk_38',
  'chunk_39',
  'chunk_40',
  'chunk_41',
  'chunk_42',
  'chunk_43',
  'chunk_44',
  'chunk_45',
  'chunk_46',
  'chunk_47',
  'chunk_48',
  'chunk_49',
  'chunk_50',
  'chunk_51',
  'chunk_52',
  'chunk_53'],
 'embeddings': None,
 'documents': ['КЛАССИФИКАТОР  \nОТДЕЛЬНЫХ КАТЕГОРИЙ ГРАЖДАН, ИМЕЮЩИХ ПРАВО НА ПОЛУЧЕНИЕ ЛЕКАРСТВЕННОЙ ПОМОЩИ  \n \n№ п\\п Код \nльготы  Категория  Закон, постановление \nправительства РФ  Процент  Перечень лекарственных сред

In [ ]:
def search_in_vector_db(query, n_results=10):
    """
    Ищет релевантные фрагменты текста по запросу.

    Args:
        query: Текстовый запрос пользователя (строка).
        n_results: Количество возвращаемых фрагментов.

    Returns:
        Список найденных текстов и их метаданных.
    """
    results = collection.query(
        query_texts=[query], # Запрос для поиска
        n_results=n_results  # Сколько фрагментов вернуть
    )

    print(results)

    # results содержит документы, расстояния и метаданные
    found_documents = results['documents'][0]  # Список текстов
    found_metadatas = results['metadatas'][0]  # Список метаданных (source, chunk_id)

    return found_documents, found_metadatas

In [ ]:
# Пример использования
query = "Какие льготы полагаются участникам войны?"
documents, metadatas = search_in_vector_db(query, n_results=10)

# Вывод результатов
for i, (doc, meta) in enumerate(zip(documents, metadatas)):
    #print(f"=== Фрагмент {i+1} (из {meta.get('source', '?')}) ===")
    print(doc)
    print()

{'ids': [['chunk_39', 'chunk_15', 'chunk_22', 'chunk_7', 'chunk_26', 'chunk_11', 'chunk_1', 'chunk_23', 'chunk_34', 'chunk_24']], 'embeddings': None, 'documents': [['взнос на капремонт.  \nДля ее оформления можно приложить \nдокументы, которые подтверждают участие лица в \nСВО (контракт добровольца или \nвоеннослужащего, повестка о призыве, справка \nиз военной части) . Если льготу получает член семьи \nмобилизованного, военнослужащего или \nдобровольца, обязательно надо представить \nдокументы, подтверждающие его статус в \nкачестве члена семьи (свидетельство о рождении, \nстраница о браке или о детях в паспорте, \nсвидетельство о браке, свидетельство о перемене \nимени). \nб) Установлен запрет ограничивать или \nприостанавливать предоставление коммунальных \nуслуг в отношении жилых помещений, где живут \nмобилизованные, контрактники, добровольцы и \nчлены их семьи. \nв) Указанные граждане вправе потребовать \nвозврата денег за купленную, но не использованную \nпутёвку если они заключ